In [1]:
import sys
from pathlib import Path

def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False

def clone_repository() -> None:
    !git clone https://github.com/featurestorebook/mlfs-book.git
    %cd mlfs-book

def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml

if is_google_colab():
    clone_repository()
    install_dependencies()
    root_dir = str(Path().absolute())
    print("Google Colab environment")
else:
    root_dir = Path().absolute()
    # Strip ~/notebooks/ccfraud from PYTHON_PATH if notebook started in one of these subdirectories
    if root_dir.parts[-1:] == ('airquality',):
        root_dir = Path(*root_dir.parts[:-1])
    if root_dir.parts[-1:] == ('notebooks',):
        root_dir = Path(*root_dir.parts[:-1])
    root_dir = str(root_dir) 
    print("Local environment")

# Add the root directory to the `PYTHONPATH` to use the `recsys` Python module from the notebook.
if root_dir not in sys.path:
    sys.path.append(root_dir)
print(f"Added the following directory to the PYTHONPATH: {root_dir}")
    
# Set the environment variables from the file <root_dir>/.env
from mlfs import config
settings = config.HopsworksSettings(_env_file=f"{root_dir}/.env")

Local environment
Added the following directory to the PYTHONPATH: /Users/wenbo/Desktop/tongxue
HopsworksSettings initialized!


<span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Daily Feature Pipeline for Air Quality (aqicn.org) and weather (openmeteo)</span>

## 🗒️ This notebook is divided into the following sections:
1. Download and Parse Data
2. Feature Group Insertion


__This notebook should be scheduled to run daily__

In the book, we use a GitHub Action stored here:
[.github/workflows/air-quality-daily.yml](https://github.com/featurestorebook/mlfs-book/blob/main/.github/workflows/air-quality-daily.yml)

However, you are free to use any Python Orchestration tool to schedule this program to run daily.

### <span style='color:#ff5f27'> 📝 Imports

In [2]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from mlfs.airquality import util
from mlfs import config
import json
import os
import warnings
warnings.filterwarnings("ignore")

## <span style='color:#ff5f27'> 🌍 Get the Sensor URL, Country, City, Street names from Hopsworks </span>

__Update the values in the cell below.__

__These should be the same values as in notebook 1 - the feature backfill notebook__


In [3]:
project = hopsworks.login()
fs = project.get_feature_store() 
secrets = hopsworks.get_secrets_api()

# This line will fail if you have not registered the AQICN_API_KEY as a secret in Hopsworks
AQICN_API_KEY = secrets.get_secret("AQICN_API_KEY").value
location_str = secrets.get_secret("SENSOR_LOCATION_JSON").value
location = json.loads(location_str)

country=location['country']
city=location['city']
street=location['street']
aqicn_url=location['aqicn_url']
latitude=location['latitude']
longitude=location['longitude']

today = datetime.date.today()

location_str

2025-11-26 05:35:20,198 INFO: Initializing external client
2025-11-26 05:35:20,198 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-11-26 05:35:21,740 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1303706


'{"country": "china", "city": "shanghai", "street": "pudonghuinan", "aqicn_url": "https://api.waqi.info/feed/@10186", "latitude": "31.2047372", "longitude": "121.4489017"}'

### <span style="color:#ff5f27;"> 🔮 Get references to the Feature Groups </span>

In [4]:
# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name='air_quality',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)

---

## <span style='color:#ff5f27'> 🌫 Retrieve Today's Air Quality data (PM2.5) from the AQI API</span>


In [5]:
import requests
import pandas as pd

aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQICN_API_KEY)
aq_today_df

,pm25,country,city,street,date,url
0,82.0,china,shanghai,pudonghuinan,2025-11-26,https://api.waqi.info/feed/@10186


In [6]:
aq_today_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   pm25     1 non-null      float32       
 1   country  1 non-null      object        
 2   city     1 non-null      object        
 3   street   1 non-null      object        
 4   date     1 non-null      datetime64[ns]
 5   url      1 non-null      object        
dtypes: datetime64[ns](1), float32(1), object(4)
memory usage: 172.0+ bytes


## <span style='color:#ff5f27'> 🌦 Get Weather Forecast data</span>

In [7]:
hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
hourly_df = hourly_df.set_index('date')

# We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
# We only want the daily weather data, so only get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = city
daily_df

Coordinates 31.25°N 121.5°E
Elevation 7.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
0,2025-11-26,9.90,0.0,8.049845,169.695206,shanghai
1,2025-11-27,7.90,0.0,9.178235,311.820068,shanghai
2,2025-11-28,8.55,0.0,8.089993,147.724380,shanghai
3,2025-11-29,13.45,0.0,9.360001,157.380096,shanghai
4,2025-11-30,15.25,0.0,11.681987,326.309906,shanghai
5,2025-12-01,10.30,0.0,7.517021,73.300674,shanghai
6,2025-12-02,13.25,0.0,9.957108,130.601212,shanghai


In [8]:
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         7 non-null      datetime64[ns]
 1   temperature_2m_mean          7 non-null      float32       
 2   precipitation_sum            7 non-null      float32       
 3   wind_speed_10m_max           7 non-null      float32       
 4   wind_direction_10m_dominant  7 non-null      float32       
 5   city                         7 non-null      object        
dtypes: datetime64[ns](1), float32(4), object(1)
memory usage: 352.0+ bytes


## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

In [9]:
# Insert new data
air_quality_fg.insert(aq_today_df)

2025-11-26 05:35:26,067 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1303706/fs/1291336/fg/1783124


Uploading Dataframe: 100.00% |█| Rows 1/1 | Elapsed Time: 00:01 | Remaining Time


Launching job: air_quality_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1303706/jobs/named/air_quality_1_offline_fg_materialization/executions


(Job('air_quality_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25",
           "min_value": -0.1,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 781313
         }
       },
       "result": {
         "observed_value": 82.0,
         "element_count": 1,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-26T04:35:26.000066Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     }
   ],
   "evaluation_parameters": {},
   "statistics": {
     "evaluated_expectations": 1,
     "successful_expectat

In [10]:
air_quality_fg.read().sort_values("date", ascending=False).head(10)   # recent 10 days

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (54.11s) 


,date,pm25,country,city,street,url
8,2025-11-18 00:00:00+00:00,62.0,china,shanghai,pudonghuinan,https://api.waqi.info/feed/@10186
699,2025-11-17 00:00:00+00:00,56.0,china,shanghai,pudonghuinan,https://api.waqi.info/feed/@10186
1223,2025-11-16 00:00:00+00:00,61.0,china,shanghai,pudonghuinan,https://api.waqi.info/feed/@10186
1242,2025-11-15 00:00:00+00:00,83.0,china,shanghai,pudonghuinan,https://api.waqi.info/feed/@10186
2156,2025-11-14 00:00:00+00:00,108.0,china,shanghai,pudonghuinan,https://api.waqi.info/feed/@10186
491,2025-11-13 00:00:00+00:00,41.0,china,shanghai,pudonghuinan,https://api.waqi.info/feed/@10186
2341,2025-11-12 00:00:00+00:00,39.0,china,shanghai,pudonghuinan,https://api.waqi.info/feed/@10186
905,2025-11-11 00:00:00+00:00,21.0,china,shanghai,pudonghuinan,https://api.waqi.info/feed/@10186
574,2025-11-10 00:00:00+00:00,67.0,china,shanghai,pudonghuinan,https://api.waqi.info/feed/@10186
1293,2025-11-09 00:00:00+00:00,60.0,china,shanghai,pudonghuinan,https://api.waqi.info/feed/@10186


In [11]:
# Grade ‘C’ tasks

# Retrieve the version-2 feature group for air quality
air_quality_fg_v2 = fs.get_feature_group(name='air_quality', version=2)

# Prepare today's core record with the required columns
today_core = aq_today_df[['date', 'pm25']].copy()
today_core['pm25'] = today_core['pm25'].astype('float32')
today_core['country'] = country
today_core['city'] = city
today_core['street'] = street
today_core['url'] = aqicn_url

# Convert date to timezone-naive datetime for consistency
today_core['date'] = pd.to_datetime(today_core['date'], utc=True).dt.tz_convert(None)
today_ts = pd.to_datetime(today)

# Read historical data from feature group (v1) and filter for the exact location
hist_v1 = air_quality_fg.read()[['date', 'country', 'city', 'street', 'pm25']].copy()
hist_v1['date'] = pd.to_datetime(hist_v1['date'], utc=True).dt.tz_convert(None)
hist_v1 = hist_v1[
    (hist_v1['country'] == country) &
    (hist_v1['city'] == city) &
    (hist_v1['street'] == street) &
    (hist_v1['date'] < today_ts)              # strictly previous days only
].sort_values('date')

# Remove duplicates on the same date, keeping the latest record
hist_v1 = hist_v1.drop_duplicates(subset=['date'], keep='last')

# Extract the last 3 PM2.5 values for lag features
last3 = hist_v1.tail(3)['pm25'].tolist()

# Ensure three lag days exist before inserting a new row
if len(last3) == 3:
    today_row = today_core.copy()
    today_row['pm25_lag1'] = last3[-1]  # yesterday
    today_row['pm25_lag2'] = last3[-2]  # two days ago
    today_row['pm25_lag3'] = last3[-3]  # three days ago

    # Ensure all PM2.5-related fields are float32
    today_row[['pm25', 'pm25_lag1', 'pm25_lag2', 'pm25_lag3']] = \
        today_row[['pm25', 'pm25_lag1', 'pm25_lag2', 'pm25_lag3']].astype('float32')

    # Match the feature group schema by filling missing columns
    fg_cols = [f.name for f in air_quality_fg_v2.features]
    for c in fg_cols:
        if c not in today_row.columns:
            today_row[c] = None
    today_row = today_row[fg_cols]

    # Insert into the feature group
    air_quality_fg_v2.insert(today_row, wait=True)
    print(f"[air_quality v2] inserted {today_ts.date()} with lag1/2/3.")
else:
    # Not enough history to compute all lag features
    print("[air_quality v2] Not enough strictly previous-day history to compute all three lags. Skipped.")


Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.03s) 
2025-11-26 05:36:39,916 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1303706/fs/1291336/fg/1783125


Uploading Dataframe: 100.00% |█| Rows 1/1 | Elapsed Time: 00:01 | Remaining Time


Launching job: air_quality_2_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1303706/jobs/named/air_quality_2_offline_fg_materialization/executions
2025-11-26 05:36:57,665 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-11-26 05:37:00,851 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-11-26 05:38:46,348 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-11-26 05:38:46,518 INFO: Waiting for log aggregation to finish.
2025-11-26 05:39:05,396 INFO: Execution finished successfully.
[air_quality v2] inserted 2025-11-26 with lag1/2/3.


In [12]:
air_quality_fg_v2.read().sort_values("date", ascending=False).head(10)   # recent 10 days


Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.53s) 


,date,pm25,country,city,street,url,pm25_lag1,pm25_lag2,pm25_lag3
2539,2025-11-26 00:00:00+00:00,82.0,china,shanghai,pudonghuinan,https://api.waqi.info/feed/@10186,62.0,56.0,61.0
11,2025-11-18 00:00:00+00:00,62.0,china,shanghai,pudonghuinan,https://api.waqi.info/feed/@10186,56.0,61.0,83.0
699,2025-11-17 00:00:00+00:00,56.0,china,shanghai,pudonghuinan,https://api.waqi.info/feed/@10186,61.0,83.0,108.0
1221,2025-11-16 00:00:00+00:00,61.0,china,shanghai,pudonghuinan,https://api.waqi.info/feed/@10186,83.0,108.0,41.0
1242,2025-11-15 00:00:00+00:00,83.0,china,shanghai,pudonghuinan,https://api.waqi.info/feed/@10186,108.0,41.0,39.0
2154,2025-11-14 00:00:00+00:00,108.0,china,shanghai,pudonghuinan,https://api.waqi.info/feed/@10186,41.0,39.0,21.0
491,2025-11-13 00:00:00+00:00,41.0,china,shanghai,pudonghuinan,https://api.waqi.info/feed/@10186,39.0,21.0,67.0
2338,2025-11-12 00:00:00+00:00,39.0,china,shanghai,pudonghuinan,https://api.waqi.info/feed/@10186,21.0,67.0,60.0
906,2025-11-11 00:00:00+00:00,21.0,china,shanghai,pudonghuinan,https://api.waqi.info/feed/@10186,67.0,60.0,83.0
574,2025-11-10 00:00:00+00:00,67.0,china,shanghai,pudonghuinan,https://api.waqi.info/feed/@10186,60.0,83.0,58.0


In [13]:
# Insert new data
weather_fg.insert(daily_df, wait=True)

2025-11-26 05:39:08,029 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1303706/fs/1291336/fg/1783126


Uploading Dataframe: 100.00% |█| Rows 7/7 | Elapsed Time: 00:01 | Remaining Time


Launching job: weather_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1303706/jobs/named/weather_1_offline_fg_materialization/executions
2025-11-26 05:39:25,677 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-11-26 05:39:32,111 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-11-26 05:41:27,761 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-11-26 05:41:27,939 INFO: Waiting for log aggregation to finish.
2025-11-26 05:41:46,833 INFO: Execution finished successfully.


(Job('weather_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "precipitation_sum",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 781316
         }
       },
       "result": {
         "observed_value": 0.0,
         "element_count": 7,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-26T04:39:08.000028Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_

## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline
 </span> 

In the following notebook you will read from a feature group and create training dataset within the feature store
